#### The concept of `phase`

Light doesn't just have brightness, but also a phase so they can add or cancel out each other

Consider two complex exponentials $e^{i\phi_1}$ and $e^{i\phi_2}$

The addition of intensity is

$$|e^{i\phi_1}+e^{i\phi_2}| = \sqrt{2+2\cos(\phi_1-\phi_2)}$$

$\cos(\phi_1-\phi_2)$ is 1 and two exponentials add up when $\phi_1-\phi_2$ is $2k\pi$, and 0 when $\phi_1-\phi_2$ is $(2k+1)\pi$

In [ ]:
import sympy as smp
import numpy as np
np.random.seed(42)
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

In [ ]:
phi_1, phi_2 = smp.symbols('phi_1 phi_2', real=True)
expr = smp.Abs(smp.exp(1j * phi_1) + smp.exp(1j * phi_2))
expr

sqrt(exp(1.0*I*phi_1)*exp(-1.0*I*phi_2) + 2 + exp(-1.0*I*phi_1)*exp(1.0*I*phi_2))

In [ ]:
simplified_expr = smp.simplify(expr.expand(trig=True))
simplified_expr

sqrt(2*cos(1.0*phi_1 - 1.0*phi_2) + 2)

#### `1D wave` equation

$$\boxed{\frac{\partial^2 \psi(x,t)}{\partial x^2}-\frac{1}{c^2}\frac{\partial^2 \psi(x,t)}{\partial^2 t^2}=0}$$

has general solution for arbitrary twice-differentiable functions $f$ and $g$

$$\psi(x,t)=f(x-ct)+g(x+ct)$$

A function with single wavelength $\lambda$, which is a special case suited for our discussion, can be written as $\cos (2\pi x/\lambda)$. To satisfy the 1D wave equation, we evaluate it at $x-ct$

$$f(x-ct)=\cos\left(\frac{2\pi}{\lambda}(x-ct) \right)=\cos(kx-\omega t)$$

where

$$\omega =kc$$

* $k=\frac{2\pi}{\lambda}$ is the `wave number`, converting `distance` $x$ to `phase` change.
It first checks how many `numbers of wavelength` fits in $x$, then each "fit" corresponds to phase change of $2\pi$

* $\omega = 2\pi f = \frac{2\pi}{T}$ is the `angular frequency`, converting `time interval` $t$ to `phase` change. It first checks how many `cycles of period` $T$ fits in $t$, then each "fit" corresponds to phase change of $2\pi$

* $c=\frac{\lambda}{T}=\frac{\omega}{k}$ is `phase velocity`, an imaginary velocity of a given point on a snapshot of wave with fixed phase. It moves a distance of a wavelength $\lambda$ during a cycle of period $T$

* For phase expressed as $kx-\omega t$, if $t$ `increases`, $x$ must `increase` to keep phase constant, so phase velocity here moves in `positive` $x$ direction

#### On `complex notation`

For a `wave` traveling in positive $x$ direction, we can write it as the `real part` of a `complex exponential`

$$\cos(kx-\omega t)=\text{Re} \left(e^{i(kx-\omega t)}\right)$$

The complex exponential allows us to factorize different components of the field

$$e^{i(kx-\omega t)}=e^{ikx}e^{-i\omega t}$$

Similarly, for a `field` in 3D, we can write

$$\begin{align*}\psi(x, y, z, t)&=A(x, y, z)\cos \left(\phi(x, y, z)-\omega t\right)\\
&=\text{Re} \left(A(x, y, z)e^{i\phi(x, y, z)}e^{-i\omega t}\right)\\
&=\text{Re} \left(U(x, y, z)e^{-i\omega t}\right)
\end{align*}$$

* $\boxed{U(x, y, z)=A(x, y, z)e^{i\phi(x, y, z)}}$ captures `non-trivial spatial dependency`, $A(x, y, z)$ often set to 1 to avoid cluttering in equations

* $e^{-i\omega t}$ is of less interest in monochromatic field where angular frequency $\omega$ does not change

* In addition, $\omega$ is commonly of high frequency, all we could ever measure is `time-averged intensity`, which is squared modulus of the `complex-valued field` (ignoring the constant)
$$I(x, y, z)=|U(x, y, z)|^2$$
so, $e^{-i\omega t}$ is (very) often dropped

#### `3D wave` equation

$$\nabla^2 \psi(x, y, z, t)-\frac{1}{c^2}\frac{\partial^2 \psi(x, y, z, t)}{\partial^2 t^2}=0$$

where the Laplacian

$$\nabla^2 = \frac{\partial^2}{\partial x^2}+\frac{\partial^2}{\partial y^2}+\frac{\partial^2}{\partial z^2}$$

If we plug in

$$\psi(x, y, z, t)=U(x, y, z)e^{-i\omega t}$$

then, we get Helmholtz equation that does `not` depend on $t$

$$\boxed{(\nabla^2+k^2)U(x, y, z)=0}$$

#### `Plane` wave

It is a solution to Helmholtz equation

$$\boxed{e^{i \textbf{k} \cdot \textbf{r}}=e^{i(k_x x +k_y y +k_z z)}}$$

* Any `position vector` $\textbf{r}$ having a `constant` dot-product with `wave vector` $\textbf{k}$ lies on a line/plane perpendicular to $\textbf{k}$ (i.e., constant phase)
* For time-dependent expression $e^{i (\textbf{k} \cdot \textbf{r} - \omega t)}$, $\textbf{k}$ indicates the `propagation direction` and line/plane of constant phase is `wavefront`
* `Wave number` $|\textbf{k}|=\frac{2\pi}{\lambda}$, and `phase velocity` $c=\frac{\omega}{|\textbf{k}|}$
* The benefits of investigating plane wave
    * The plane wave formula indicates that it is related to `Fourier transform` and can be used to decompose a field
    * A spherical wave emitted by a point source can be approximated by a plane wave in far field

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Arc, Rectangle, Ellipse, Circle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
plt.style.use('dark_background')

In [ ]:
class PlaneWave:
    def __init__(self, kx, ky, omega, dt, duration, direction='positive'):
        if kx == 0 and ky == 0:
            raise ValueError('k must be non-zero')

        if direction not in ['positive', 'negative']:
            raise ValueError('direction must be either "positive" or "negative"')

        self.kx = kx
        self.ky = ky
        self.omega = omega
        self.dt = dt
        self.duration = duration
        self.direction = direction


        self.x = np.linspace(-3*np.pi, 3*np.pi, 300)
        self.y = np.linspace(-3*np.pi, 3*np.pi, 300)
        self.xx, self.yy = np.meshgrid(self.x, self.y)

        self.k_squared = kx**2 + ky**2
        self.k_scale = 2*np.pi/self.k_squared

    def plot_wave(self, time):
        """handles the actual wave plotting at a given time"""
        # compute wave value at each (x, y)
        if self.direction == 'positive':
            self.wave = (np.exp(1j * (self.kx * self.xx + self.ky * self.yy - self.omega * time))).real
        else:
            self.wave = (np.exp(1j * (self.kx * self.xx + self.ky * self.yy + self.omega * time))).real

        self.ax.clear()
        self.levels = np.linspace(self.wave.min(), self.wave.max(), 100)
        self.c = self.ax.contourf(self.xx, self.yy, self.wave, levels=self.levels, cmap='Blues')
        if not hasattr(self, 'colorbar'):  # Only add colorbar once
            self.colorbar = self.fig.colorbar(self.c, ax=self.ax, label='Wave Amplitude')

        self.ax.set_aspect('equal', adjustable='box')
        self.ax.set_xlabel('x')
        self.ax.set_ylabel('y')
        self.ax.set_title(f'$\Delta t$: {time:.2f} s, $\omega$: {self.omega} rad/s, $k_x$: {self.kx}, $k_y$: {self.ky} \n temporal phase shift: {self.omega*time/(2*np.pi):.2f} cycle(s)')

        self.ax.quiver(0, 0, self.kx*self.k_scale, self.ky*self.k_scale, angles='xy', scale_units='xy', scale=1, color='r', width=0.01, alpha=0.8, label='$\mathbf{k}$, scaled to $\lambda$')
        self.ax.legend()
        # self.ax.grid(True)
        self.fig.tight_layout()

    def main(self, animation=False):
        """main method to initialize plot and decides whether to animate or just show static plot."""
        self.animation = animation
        self.fig, self.ax = plt.subplots(figsize=(6, 5)) # this only needs setup once
        self.plot_wave(0) # plot at time 0 if not otherwise specified

        if self.animation:
            return self.animate() # this calls self.animate() and returns HTML object
        else:
            plt.show()

    def update(self, frame):
        """used for updating the plot in each frame during the animation"""
        self.plot_wave(frame)

    def animate(self):
        """set up and return the animation"""
        self.num_frames = int(self.duration / self.dt)
        self.ani = FuncAnimation(self.fig, self.update, frames=np.linspace(0, self.duration, self.num_frames), blit=False)
        plt.close(self.fig)
        return HTML(self.ani.to_jshtml())

In [ ]:
plane_wave = PlaneWave(kx=1, ky=2, omega=1, dt=0.25, duration=2*np.pi, direction='positive')
plane_wave.main(animation=True)